# Great Barrier Reef Identification using ScaledYOLOv4

In [ ]:
# CONTROL PANEL

MODEL_TYPE = 'P5' # can be one of 'P5', 'P6', or 'P7'

NUM_EPOCHS = 6

# Training resolution
RESOLUTION = 1280

# Depends on GPU RAM which depends on training resolution and model type
# - suggest to run in interactive mode to determine the largest batch size that does not result in CUDA out of memory error
BATCH_SIZE = 4

In [ ]:
# Module imports

import matplotlib.pyplot as plt
import shutil

## Import Data prepped in YOLO format
* Data was prepared using this notebook: https://www.kaggle.com/alexchwong/cots-data-prep-fold4

In [ ]:
DATA_DIR  = '/kaggle/input/cots-data-prep-fold4'

In [ ]:
!unzip -qq -d /kaggle/working/images {DATA_DIR}/images.zip 
!unzip -qq -d /kaggle/working/labels {DATA_DIR}/labels.zip
!cp {DATA_DIR}/cots.yaml /kaggle/working/

## Installing ScaledYOLOv4 for Kaggle

In [ ]:
!cp -r /kaggle/input/scaledyolov4-installation/ScaledYOLOv4 /kaggle/working/
!cp -r /kaggle/input/scaledyolov4-installation/mish-cuda /kaggle/working/

### Mish-CUDA (must enable GPU)

In [ ]:
# mish-cuda is required in order to use the pre-trained models

%cd /kaggle/working/mish-cuda
!python setup.py build install

## Fixes version `GLIBCXX_3.4.26' not found on Kaggle

In [ ]:
# Yes, this takes about 5 minutes

!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt-get update
!apt-get upgrade libstdc++6 -y

## Hyperparameters
* Essentially the same as YOLOv5

In [ ]:
hyps = '''
# these settings are the same as hyp.finetune.yaml

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
giou: 0.05  # GIoU loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.5  # image translation (+/- fraction)
scale: 0.8  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mixup: 0.2  # image mixup (probability)
'''

with open('/kaggle/working/ScaledYOLOv4/data/hyp.custom.yaml', 'w') as f:
    f.write(hyps)

!cat /kaggle/working/ScaledYOLOv4/data/hyp.custom.yaml

### Checking YAML file for dataset

In [ ]:
!cat /kaggle/working/cots.yaml

### Defining Model for one Object

* Edit the number of classes to 1

In [ ]:
if MODEL_TYPE == 'P5':
    MODEL_YAML = 'yolov4-p5.yaml'
    MODEL_PATH = '/kaggle/input/scaledyolov4-installation/ScaledYOLOv4_pretrained/yolov4_p5.pt'
elif MODEL_TYPE == 'P6':
    MODEL_YAML = 'yolov4-p6.yaml'
    MODEL_PATH = '/kaggle/input/scaledyolov4-installation/ScaledYOLOv4_pretrained/yolov4_p6.pt'
elif MODEL_TYPE == 'P7':
    MODEL_YAML = 'yolov4-p7.yaml'
    MODEL_PATH = '/kaggle/input/scaledyolov4-installation/ScaledYOLOv4_pretrained/yolov4_p7.pt'

In [ ]:
# edit number of classes to 1

shutil.copy(f"/kaggle/working/ScaledYOLOv4/models/{MODEL_YAML}", f"/kaggle/working/ScaledYOLOv4/{MODEL_YAML}")   
!sed -i 's/nc: 80/nc: 1/g' /kaggle/working/ScaledYOLOv4/{MODEL_YAML}
!cat /kaggle/working/ScaledYOLOv4/{MODEL_YAML}

# Training ScaledYOLOv4

### Run training script

In [ ]:
%cd /kaggle/working/ScaledYOLOv4
!wandb disabled
!python train.py \
    --epochs {NUM_EPOCHS} \
    --save-epochs 3 \
    --hyp /kaggle/working/ScaledYOLOv4/data/hyp.custom.yaml \
    --batch-size {BATCH_SIZE} \
    --img {RESOLUTION} {RESOLUTION} \
    --data /kaggle/working/cots.yaml \
    --cfg /kaggle/working/ScaledYOLOv4/{MODEL_YAML} \
    --weights {MODEL_PATH} \
    --name cots

# save-epochs: How many last epochs to save (i.e. setting this to 3 will save epochs n, n-1, and n-2)

### Checking Model Performance

In [ ]:
TRAIN_RUN_DIR  = '/kaggle/working/ScaledYOLOv4/runs/exp0_cots'
!ls {TRAIN_RUN_DIR}

In [ ]:
!printf 'epoch\tGPU\tgIOU\tobj\tcls\ttotal\tnTarg\tres\tP\tR\tmAP50\tmAP50:95\tv_gIOU\tv_obj\tv_cls\n'
!cat {TRAIN_RUN_DIR}/results.txt

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread(f'{TRAIN_RUN_DIR}/results.png'));

# Cleanup

In [ ]:
!mv {TRAIN_RUN_DIR} /kaggle/working/

In [ ]:
%cd /kaggle/working

!rm -r /kaggle/working/ScaledYOLOv4
!rm -r /kaggle/working/mish-cuda

!rm -r /kaggle/working/images
!rm -r /kaggle/working/labels

# Credits:
https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer